In [474]:
import pandas as pd
import os
from datetime import datetime

In [475]:
events = []


def parse_time(time_string):
    return datetime.strptime(time_string, "%M:%S,%f")


def event(player, event_name, time_string):
    return {"player": player, "event": event_name, "timestamp": parse_time(time_string)}

def player_id(player_string):
    return int(player_string.replace("<Files\\\\Player ", "").split(">")[0])

In [476]:
file = open('../Codes/Chamber.txt', 'r', encoding="utf16", errors='ignore')
lines = file.readlines()
for line in lines:
    if "Player" in line:
        current_player = player_id(line)
    elif line[0] == '[':
        time_strings = line.replace("[", "").replace("]", "").replace("\n", "").split(" - ")
        events.append(event(current_player, "chamber_enter", time_strings[0]))
        events.append(event(current_player, "chamber_exit", time_strings[1]))

In [477]:
file = open('../Codes/Death.txt', 'r', encoding="utf16", errors='ignore')
lines = file.readlines()
for line in lines:
    if "Player" in line:
        current_player = player_id(line)
    elif line[0] == '[':
        time_strings = line.replace("[", "").replace("]", "").replace("\n", "").split(" - ")
        events.append(event(current_player, "death", time_strings[0]))

In [478]:
file = open('../Codes/Meg Encounter.txt', 'r', encoding="utf16", errors='ignore')
lines = file.readlines()
for line in lines:
    if "Player" in line:
        current_player = player_id(line)
    elif line[0] == '[':
        time_strings = line.replace("[", "").replace("]", "").replace("\n", "").split(" - ")
        events.append(event(current_player, "boss_fight_begin", time_strings[0]))
        events.append(event(current_player, "boss_fight_end", time_strings[1]))

In [479]:
file = open('../Codes/Run.txt', 'r', encoding="utf16", errors='ignore')
lines = file.readlines()
for line in lines:
    if "Player" in line:
        current_player = player_id(line)
    elif line[0] == '[':
        time_strings = line.replace("[", "").replace("]", "").replace("\n", "").split(" - ")
        events.append(event(current_player, "run_begin", time_strings[0]))
        events.append(event(current_player, "run_end", time_strings[1]))

In [480]:
dataset = pd.DataFrame(events).sort_values(by=["player", "timestamp"])
dataset["chamber"] = None
dataset["attempt"] = None

attempt=0
in_run=False
chamber=None
player=1
death=False
for index, row in dataset.iterrows():
    if player < row["player"]:
        player +=1
        attempt = 0
        chamber = None

    if row["event"] == "chamber_enter":
        if in_run is False:
            in_run = True
            chamber = 1
            attempt += 1
            death = False
        elif chamber is None:
            chamber = 1
        else:
            chamber +=1

    if row["event"] == "run_begin":
        if in_run is False:
            in_run = True
            death = False
            chamber = None
            attempt += 1

    if row["event"] == "death":
        death = True

    if row["event"] == "chamber_exit" and chamber is None:
        chamber = 1

    dataset.at[index,'chamber'] = chamber
    dataset.at[index,'attempt'] = attempt

    if row["event"] == "run_end":
        in_run = False

dataset

,player,event,timestamp,chamber,attempt
1278,1,run_begin,1900-01-01 00:01:50.100,None,1
0,1,chamber_enter,1900-01-01 00:02:09.100,1,1
1,1,chamber_exit,1900-01-01 00:03:57.000,1,1
2,1,chamber_enter,1900-01-01 00:04:00.400,2,1
3,1,chamber_exit,1900-01-01 00:04:48.600,2,1
...,...,...,...,...,...
765,20,chamber_exit,1900-01-01 00:35:21.500,5,7
766,20,chamber_enter,1900-01-01 00:35:23.900,6,7
1232,20,death,1900-01-01 00:35:58.900,6,7
767,20,chamber_exit,1900-01-01 00:35:59.100,6,7


In [481]:
dataset.to_csv("../Processed/game_events.csv", index=False, date_format="%M:%S:%f")

In [482]:
def player_stats(player):
    return pd.Series({"time":(player["timestamp"].max()-player["timestamp"].min()).total_seconds(),
                      "total_chambers":player["event"][player["event"] == "chamber_enter"].count(),
                      "max_chamber":player["chamber"].max(),
                      "mean_chamber_per_run":player["event"][player["event"] == "chamber_enter"].count()/player["attempt"].max(),
                      "number_of_runs":player["attempt"].max(),
                      "boss_encounters":player["event"][player["event"] == "boss_fight_begin"].count()
                      })

dataset.groupby(["player"]).apply(player_stats).reset_index().to_csv("../Processed/player_stats.csv", index=False, date_format="%M:%S:%f")